In [2]:
import logging
import sys
sys.path.append('/remote/gpu02/crescenzo/MadMiner')
from madminer.lhe import LHEReader
from madminer.sampling import SampleAugmenter
from madminer import sampling

In [3]:
# MadMiner output
logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [4]:
from madminer import MadMiner
import numpy as np

In [5]:
miner = MadMiner()

miner.load("data/setup.h5")

15:17 madminer.utils.inter INFO    HDF5 file does not contain nuisance parameters information
15:17 madminer.utils.inter INFO    HDF5 file does not contain finite difference information
15:17 madminer.utils.inter INFO    HDF5 file does not contain observables information
15:17 madminer.utils.inter INFO    HDF5 file does not contain sample summary information
15:17 madminer.utils.inter INFO    HDF5 file does not contain sample information
15:17 madminer.utils.inter INFO    HDF5 file does not contain systematic information
15:17 madminer.core.madmin INFO    Found 2 parameters:
15:17 madminer.core.madmin INFO       CWL2 (LHA: dim6 2, Power: 2, Range: (0.0, 1.0))
15:17 madminer.core.madmin INFO       CPWL2 (LHA: dim6 5, Power: 2, Range: (0.0, 1.0))
15:17 madminer.core.madmin INFO    Found 6 benchmarks:
15:17 madminer.core.madmin INFO       SM: CWL2 = 0.00e+00, CPWL2 = 0.00e+00
15:17 madminer.core.madmin INFO       morphing_basis_vector_1: CWL2 = 0.93, CPWL2 = 1.48
15:17 madminer.core.madmi

In [32]:
list(miner.benchmarks.keys())[0]

'SM'

In [13]:
mg_dir = "/remote/gpu02/crescenzo/MG5_aMC_v3_5_3"
for j,name in enumerate(list(miner.benchmarks.keys())):
    if j == 0:
        print(name)
        miner.run(
        sample_benchmark=name,
        mg_directory=mg_dir,
        mg_process_directory=f"./mg_processes/signal_{j}",
        proc_card_file="cards/proc_card_signal.dat",
        param_card_template_file="cards/param_card_template.dat",
        run_card_file="cards/run_card_signal_large.dat",
        log_directory=f"logs/signal_{j}",
        python_executable="python3",
        only_prepare_script=True
    )

15:07 madminer.core.madmin INFO    Adding python3 bin folder to PATH
15:07 madminer.core.madmin INFO    Using Python executable /remote/gpu02/crescenzo/environment/bin/python3
15:07 madminer.utils.inter INFO    Generating MadGraph process folder from cards/proc_card_signal.dat at ./mg_processes/signal_0
15:07 madminer.utils.inter INFO    Calling MadGraph: export PATH=/remote/gpu02/crescenzo/environment/bin:$PATH; python3 /remote/gpu02/crescenzo/MG5_aMC_v3_5_3/bin/mg5_aMC /tmp/generate.mg5


SM


15:07 madminer.core.madmin INFO    Run 0
15:07 madminer.core.madmin INFO      Sampling from benchmark: SM
15:07 madminer.core.madmin INFO      Original run card:       cards/run_card_signal_large.dat
15:07 madminer.core.madmin INFO      Original Pythia8 card:   None
15:07 madminer.core.madmin INFO      Original MadSpin card:   None
15:07 madminer.core.madmin INFO      Original config card:    None
15:07 madminer.core.madmin INFO      Copied run card:         madminer/cards/run_card_0.dat
15:07 madminer.core.madmin INFO      Copied Pythia8 card:     None
15:07 madminer.core.madmin INFO      Copied MadSpin card:     None
15:07 madminer.core.madmin INFO      Copied config card:      None
15:07 madminer.core.madmin INFO      Param card:              madminer/cards/param_card_0.dat
15:07 madminer.core.madmin INFO      Reweight card:           madminer/cards/reweight_card_0.dat
15:07 madminer.core.madmin INFO      Log file:                run_0.log
15:07 madminer.core.madmin INFO    Creating

In [6]:
cross_sections = []

for j,name in enumerate(list(miner.benchmarks.keys())):
    lhe = LHEReader("data/setup.h5")
    lhe.add_sample(
                lhe_filename=f"mg_processes/signal_{j}/Events/run_01/unweighted_events.lhe.gz",
                sampled_from_benchmark=name,
                is_background=False,
                k_factor=1.0,
            )
    lhe.add_observable(
            "met",
            "met.pt",
            required=True,
        )
    lhe.analyse_samples()
    lhe.save(f"data/lhe_data_{j}.h5")
    sampler = SampleAugmenter(f"data/lhe_data_{j}.h5")
    cross_sections.append(sampler.cross_sections(sampling.benchmark(name))[1][0])



15:17 madminer.utils.inter INFO    HDF5 file does not contain nuisance parameters information
15:17 madminer.utils.inter INFO    HDF5 file does not contain finite difference information
15:17 madminer.utils.inter INFO    HDF5 file does not contain observables information
15:17 madminer.utils.inter INFO    HDF5 file does not contain sample summary information
15:17 madminer.utils.inter INFO    HDF5 file does not contain sample information
15:17 madminer.utils.inter INFO    HDF5 file does not contain systematic information
15:17 madminer.lhe.lhe_rea INFO    Analysing LHE sample mg_processes/signal_0/Events/run_01/unweighted_events.lhe.gz: Calculating 1 observables, requiring 0 selection cuts, using 0 efficiency factors, associated with no systematics


Parallelize:  False


15:19 madminer.lhe.lhe_rea INFO    Analysed number of events per sampling benchmark:
15:19 madminer.lhe.lhe_rea INFO      10000 from SM
15:19 madminer.analysis.da INFO    Loading data from data/lhe_data_0.h5
15:19 madminer.utils.inter INFO    HDF5 file does not contain nuisance parameters information
15:19 madminer.utils.inter INFO    HDF5 file does not contain finite difference information
15:19 madminer.utils.inter INFO    HDF5 file does not contain systematic information


ValueError: too many values to unpack (expected 14)

In [12]:
##JUST EXAMPLE!
##SUPPOSE I HAVE MAX POWER = 2 -> 6 BENCHMARK POINTS
cs_from_madgraph = np.linspace(0.001,0.010,6)
reduced_cs = cs_from_madgraph[1:cs_from_madgraph.size]/cs_from_madgraph[0]
##NOTICE THE NEW SIZE IS N_BENCHMARKS - 1

miner_2.morpher.set_reduced(reduced_cs)

In [13]:
miner_2.morpher.reduced_cs

array([ 2.8,  4.6,  6.4,  8.2, 10. ])

Now we have all ingredients to set up the morphing ratio

In [28]:
miner.ratio_set_morphing(max_overall_power=2)
miner.save("saving_file") 

11:53 madminer.core.madmin INFO    Optimizing basis for morphing the ratio
11:53 madminer.core.madmin WARNING Reset morphing
11:53 madminer.core.madmin INFO    Testing out
11:53 madminer.core.madmin INFO    Saving setup (including morphing) to saving_file


Now I have decided the actual benchmark points. One needs to run again a simulation at each of this (can skip the SM one since it was already covered during the cs morphing optimizaiton)

In [35]:
for j,elem in enumerate(miner.morpher.basis):
    #